In [10]:
import requests
import pandas as pd
import json
import uuid
import time
import glob
import ipywidgets as widgets

## Custom functions

In [2]:
def extract_all_trials(records, format="pandas"):
  # stack all trials for all records
  all_trials = []
  for cur_record in range(0,len(records)):
    trial_content = json.loads(records[cur_record]['content'])
    trial_data = trial_content.get('trials',[])
    all_trials.extend(trial_data)

  if format == "pandas":
    # create pandas dataframe from all trials
    all_trials_df = pd.DataFrame(all_trials)
    return all_trials_df
  else:
    return all_trials

In [3]:
def extract_all_otherstuff(records, format="pandas"):
  # stack all trials for all records
  all_trials = []
  for cur_record in range(0,len(records)):
    trial_content = json.loads(records[cur_record]['content'])
    trial_data = trial_content.get('trials',[])
    all_trials.extend(trial_data)

  if format == "pandas":
    # create pandas dataframe from all trials
    all_trials_df = pd.DataFrame(all_trials)
    return all_trials_df
  else:
    return all_trials

## Run Job

In [108]:
def summary_symbol_search(x, trials_expected=20):
    d = {}
    d['flag_is_invalid_n_trials'] = x['session_uuid'].count() != trials_expected
    d['n_trials'] = x['session_uuid'].count()
    d['n_trials_lure'] = (x['trial_type'] == "lure").sum()
    d['n_trials_responsetime_lt250ms'] = (
        x['response_time_duration_ms'] < 250).sum()
    d['n_trials_responsetime_gt10000ms'] = (
        x['response_time_duration_ms'] > 10000).sum()
    d['n_correct_trials'] = (x['user_response_index']
                             == x['correct_response_index']).sum()
    d['n_incorrect_trials'] = (x['user_response_index']
                             != x['correct_response_index']).sum()
    d['mean_response_time_overall'] = x['response_time_duration_ms'].mean()
    d['mean_response_time_correct'] = x.loc[(x['user_response_index']
                                             == x['correct_response_index']), 'response_time_duration_ms'].mean()
    d['mean_response_time_incorrect'] = x.loc[(x['user_response_index']
                                               != x['correct_response_index']), 'response_time_duration_ms'].mean()
    d['median_response_time_overall'] = x['response_time_duration_ms'].median()
    d['median_response_time_correct'] = x.loc[(x['user_response_index']
                                               == x['correct_response_index']), 'response_time_duration_ms'].median()
    d['median_response_time_incorrect'] = x.loc[(x['user_response_index']
                                                 != x['correct_response_index']), 'response_time_duration_ms'].median()
    d['sd_response_time_overall'] = x['response_time_duration_ms'].std()
    d['sd_response_time_correct'] = x.loc[(x['user_response_index']
                                           == x['correct_response_index']), 'response_time_duration_ms'].std()
    d['sd_response_time_incorrect'] = x.loc[(x['user_response_index']
                                             != x['correct_response_index']), 'response_time_duration_ms'].std()
    return pd.Series(d,
                     index=['flag_is_invalid_n_trials',
                            # 'flag_is_potentially_invalid_rt',
                            'n_trials',
                            'n_trials_lure',
                            'n_correct_trials',
                            'n_incorrect_trials',
                            'n_trials_responsetime_lt250ms',
                            'n_trials_responsetime_gt10000ms',
                            'mean_response_time_overall',
                            'mean_response_time_correct',
                            'mean_response_time_incorrect',
                            'median_response_time_overall',
                            'median_response_time_correct',
                            'median_response_time_incorrect',
                            'sd_response_time_overall',
                            'sd_response_time_correct',
                            'sd_response_time_incorrect'
                            ])

def summary_grid_memory(x, trials_expected=4):
    d = {}
    d['flag_is_invalid_n_trials'] = x['session_uuid'].count() != trials_expected
    d['n_trials'] = x['session_uuid'].count()
    # d['n_trials_responsetime_lt250ms'] = (
    #     x['response_time_duration_ms'] < 250).sum()
    # d['n_trials_responsetime_gt10000ms'] = (
    #     x['response_time_duration_ms'] > 10000).sum()
    d['n_perfect_trials'] = (x['number_of_correct_dots'] == 3.0).sum()
    #d['n_user_interference_actions'] = len(json.loads(x['user_interference_actions'].values[0]))
    d['mean_correct_dots'] = (x['number_of_correct_dots']).mean()
    d['min_correct_dots'] = (x['number_of_correct_dots']).min()
    d['sum_correct_dots'] = (x['number_of_correct_dots']).sum()
    #d['mean_response_time_overall'] = x['response_time_duration_ms'].mean()
    # d['mean_response_time_correct'] = x.loc[(x['user_response_index']
    #                                          == x['correct_response_index']), 'response_time_duration_ms'].mean()
    #d['median_response_time_overall'] = x['response_time_duration_ms'].median()
    #d['sd_response_time_overall'] = x['response_time_duration_ms'].std()
    return pd.Series(d,
                     index=['flag_is_invalid_n_trials',
                            # 'flag_is_potentially_invalid_rt',
                            'n_trials',
                            # 'n_trials_responsetime_lt250ms',
                            # 'n_trials_responsetime_gt10000ms',
                            'n_perfect_trials',
                            # 'n_user_interference_actions',
                            'mean_correct_dots',
                            'min_correct_dots',
                            'sum_correct_dots',
                            # 'mean_response_time_overall',
                            # 'median_response_time_overall',
                            # 'sd_response_time_overall',
                            ])


In [151]:
import pandas as pd

#https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html
data_files = glob.glob('data/*.json')
file_to_process = widgets.Dropdown(
    options=data_files,
    layout={'width': 'max-content'},
    value=(data_files[0]),
    description='File to convert:',
)
display(file_to_process)


Dropdown(description='File to convert:', layout=Layout(width='max-content'), options=('data/2023-01-20_05-56-3…

In [152]:
# constants
fn_split_delim = "-data_study-" # delimiter for splitting filename

# generate job timestamp and uuid
job_uuid = str(uuid.uuid4())
job_ts = time.strftime("%Y%m%d-%H%M%S")

# load data from exporter S3 bucket
fn = file_to_process.value
all_records = json.load(open(fn))

# extract study_uid from filename
study_uid = fn.split(fn_split_delim)[1].split(".")[0] # the thing right after the split delimiter
print(f"Processing study: {study_uid} | {fn}")

# extract 'trials' from records object
all_trials = extract_all_trials(all_records, format="pandas")
display(all_trials)

raw_csv_file = f"tidy/{job_ts}_m2c2kit_exporter-raw-data_uuid-{job_uuid}_study-{study_uid}.csv"

# save to csv
all_trials.to_csv(raw_csv_file, index=False)

df = pd.read_csv(raw_csv_file)
df.columns
display(df.head(3))

# number of sessions per activity
df.groupby("activity_id")["session_uuid"].count()

# number of api_keys = should be 1 per study
df.groupby("api_key")["api_key"].count()

for activity, frame in df.groupby("activity_id"):
    print(f"Processing activity: {activity}")
    print("-------------------------")
    frame_ = frame.drop_duplicates()
    frame_.columns
    csv_file_act = f"tidy/{job_ts}_m2c2kit_exporter-summary-data_activity-{activity}_uuid-{job_uuid}_study-{study_uid}.csv"
    if(activity == "symbol-search"):
        summary_df = frame_.groupby(["participant_id", "session_uuid", "session_id"]).apply(summary_symbol_search)
        summary_df.reset_index().to_csv(csv_file_act, index=False)
        display(summary_df)
    else:
        summary_df = frame_.groupby(["participant_id", "session_uuid", "session_id"]).apply(summary_grid_memory)
        summary_df.reset_index().to_csv(csv_file_act, index=False)
    display(summary_df)

data/2023-01-20_05-55-51-db-m2c2api_collection-data_study-MOTLAB.json
Processing study: MOTLAB


,session_uuid,activity_uuid,activity_id,activity_version,activity_begin_iso8601_timestamp,trial_begin_iso8601_timestamp,trial_index,presented_cells,selected_cells,user_dot_actions,...,device_metadata,session_id,study_id,participant_id,api_key,trial_type,card_configuration,response_time_duration_ms,user_response_index,correct_response_index
0,7bdb246e-bbf9-4a0a-8b54-0026336cb5ee,e9ba8871-3395-4f83-ba01-78136160a914,grid-memory,0.8.1,2023-01-15T18:23:25.139Z,2023-01-15T18:23:52.964Z,0,"[{'row': 2, 'column': 3}, {'row': 1, 'column':...","[{'row': 1, 'column': 0}, {'row': 3, 'column':...","[{'elapsed_duration_ms': 2193.199999988079, 'a...",...,{'userAgent': 'Mozilla/5.0 (Windows NT 10.0; W...,2,MOTLAB,85874,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
1,7bdb246e-bbf9-4a0a-8b54-0026336cb5ee,e9ba8871-3395-4f83-ba01-78136160a914,grid-memory,0.8.1,2023-01-15T18:23:25.139Z,2023-01-15T18:24:11.374Z,1,"[{'row': 1, 'column': 3}, {'row': 0, 'column':...","[{'row': 3, 'column': 4}, {'row': 2, 'column':...","[{'elapsed_duration_ms': 6042.300000011921, 'a...",...,{'userAgent': 'Mozilla/5.0 (Windows NT 10.0; W...,2,MOTLAB,85874,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
2,7bdb246e-bbf9-4a0a-8b54-0026336cb5ee,e9ba8871-3395-4f83-ba01-78136160a914,grid-memory,0.8.1,2023-01-15T18:23:25.139Z,2023-01-15T18:24:32.481Z,2,"[{'row': 0, 'column': 4}, {'row': 0, 'column':...","[{'row': 0, 'column': 4}, {'row': 1, 'column':...","[{'elapsed_duration_ms': 1389.5, 'action_type'...",...,{'userAgent': 'Mozilla/5.0 (Windows NT 10.0; W...,2,MOTLAB,85874,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
3,7bdb246e-bbf9-4a0a-8b54-0026336cb5ee,e9ba8871-3395-4f83-ba01-78136160a914,grid-memory,0.8.1,2023-01-15T18:23:25.139Z,2023-01-15T18:24:51.722Z,3,"[{'row': 4, 'column': 2}, {'row': 1, 'column':...","[{'row': 0, 'column': 3}, {'row': 1, 'column':...","[{'elapsed_duration_ms': 2340.800000011921, 'a...",...,{'userAgent': 'Mozilla/5.0 (Windows NT 10.0; W...,2,MOTLAB,85874,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
4,7bdb246e-bbf9-4a0a-8b54-0026336cb5ee,e9ba8871-3395-4f83-ba01-78136160a914,grid-memory,0.8.1,2023-01-15T18:23:25.139Z,2023-01-15T18:23:52.964Z,0,"[{'row': 2, 'column': 3}, {'row': 1, 'column':...","[{'row': 1, 'column': 0}, {'row': 3, 'column':...","[{'elapsed_duration_ms': 2193.199999988079, 'a...",...,{'userAgent': 'Mozilla/5.0 (Windows NT 10.0; W...,2,MOTLAB,85874,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572931,b18d52f8-aa95-44e6-b550-7df7860fd14f,6b707cdb-2edc-48e3-9d28-916e57c3c551,grid-memory,0.8.1,2022-12-16T15:30:29.946Z,2022-12-16T15:30:41.267Z,0,"[{'row': 3, 'column': 1}, {'row': 0, 'column':...","[{'row': 4, 'column': 1}, {'row': 2, 'column':...","[{'elapsed_duration_ms': 1882.7999999821186, '...",...,{'userAgent': 'Mozilla/5.0 (Linux; Android 12;...,2,MOTLAB,82061,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
572932,b18d52f8-aa95-44e6-b550-7df7860fd14f,6b707cdb-2edc-48e3-9d28-916e57c3c551,grid-memory,0.8.1,2022-12-16T15:30:29.946Z,2022-12-16T15:31:00.247Z,1,"[{'row': 4, 'column': 0}, {'row': 0, 'column':...","[{'row': 0, 'column': 0}, {'row': 4, 'column':...","[{'elapsed_duration_ms': 2172.199999988079, 'a...",...,{'userAgent': 'Mozilla/5.0 (Linux; Android 12;...,2,MOTLAB,82061,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
572933,b18d52f8-aa95-44e6-b550-7df7860fd14f,6b707cdb-2edc-48e3-9d28-916e57c3c551,grid-memory,0.8.1,2022-12-16T15:30:29.946Z,2022-12-16T15:30:41.267Z,0,"[{'row': 3, 'column': 1}, {'row': 0, 'column':...","[{'row': 4, 'column': 1}, {'row': 2, 'column':...","[{'elapsed_duration_ms': 1882.7999999821186, '...",...,{'userAgent': 'Mozilla/5.0 (Linux; Android 12;...,2,MOTLAB,82061,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
572934,b18d52f8-aa95-44e6-b550-7df7860fd14f,6b707cdb-2edc-48e3-9d28-916e57c3c551,grid-memory,0.8.1,2022-12-16T15:30:29.946Z,2022-12

/Users/nelsonroque/.pyenv/versions/3.7.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,session_uuid,activity_uuid,activity_id,activity_version,activity_begin_iso8601_timestamp,trial_begin_iso8601_timestamp,trial_index,presented_cells,selected_cells,user_dot_actions,...,device_metadata,session_id,study_id,participant_id,api_key,trial_type,card_configuration,response_time_duration_ms,user_response_index,correct_response_index
0,7bdb246e-bbf9-4a0a-8b54-0026336cb5ee,e9ba8871-3395-4f83-ba01-78136160a914,grid-memory,0.8.1,2023-01-15T18:23:25.139Z,2023-01-15T18:23:52.964Z,0,"[{'row': 2, 'column': 3}, {'row': 1, 'column':...","[{'row': 1, 'column': 0}, {'row': 3, 'column':...","[{'elapsed_duration_ms': 2193.199999988079, 'a...",...,{'userAgent': 'Mozilla/5.0 (Windows NT 10.0; W...,2,MOTLAB,85874,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
1,7bdb246e-bbf9-4a0a-8b54-0026336cb5ee,e9ba8871-3395-4f83-ba01-78136160a914,grid-memory,0.8.1,2023-01-15T18:23:25.139Z,2023-01-15T18:24:11.374Z,1,"[{'row': 1, 'column': 3}, {'row': 0, 'column':...","[{'row': 3, 'column': 4}, {'row': 2, 'column':...","[{'elapsed_duration_ms': 6042.300000011921, 'a...",...,{'userAgent': 'Mozilla/5.0 (Windows NT 10.0; W...,2,MOTLAB,85874,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN
2,7bdb246e-bbf9-4a0a-8b54-0026336cb5ee,e9ba8871-3395-4f83-ba01-78136160a914,grid-memory,0.8.1,2023-01-15T18:23:25.139Z,2023-01-15T18:24:32.481Z,2,"[{'row': 0, 'column': 4}, {'row': 0, 'column':...","[{'row': 0, 'column': 4}, {'row': 1, 'column':...","[{'elapsed_duration_ms': 1389.5, 'action_type'...",...,{'userAgent': 'Mozilla/5.0 (Windows NT 10.0; W...,2,MOTLAB,85874,e482b3a1-3b71-4883-a4c2-6efab9db69c6,NaN,NaN,NaN,NaN,NaN


Processing activity: grid-memory
-------------------------


flag_is_invalid_n_trials  \
participant_id session_uuid                         session_id                             
1025           89d0639e-75de-4b23-8a73-105e47608e52 2                              False   
               b214346e-66ce-4474-a413-9bd5ec992ec3 1                              False   
1053           87aafffd-97d7-4510-99b5-a3ae0de45e69 1                              False   
               d9b9baaf-3416-4677-a3e1-b421c8220c53 2                              False   
1059           3d621016-fe1c-4b3c-8858-7e2a7cf550a1 2                              False   
...                                                                                  ...   
QUALTRICSID    69611119-0775-4984-be4c-0f82a2984495 ANUMBER                        False   
               796952b7-b967-4bf2-b778-14baec6e3382 ANUMBER                        False   
               ae12f15f-b1fd-4eca-9f20-b938e3509745 ANUMBER                        False   
               bb8fdcc8-5735-42e8-a136-762f4407a6a5 ANUMBER                         True   
               f60bb472-4afc-4903-836c-f8eacc686631 ANUMBER                        False   

                                                                n_trials  \
participant_id session_uuid                         session_id             
1025           89d0639e-75de-4b23-8a73-105e47608e52 2                  4   
               b214346e-66ce-4474-a413-9bd5ec992ec3 1                  4   
1053           87aafffd-97d7-4510-99b5-a3ae0de45e69 1                  4   
               d9b9baaf-3416-4677-a3e1-b421c8220c53 2                  4   
1059           3d621016-fe1c-4b3c-8858-7e2a7cf550a1 2                  4   
...                                                                  ...   
QUALTRICSID    69611119-0775-4984-be4c-0f82a2984495 ANUMBER            4   
               796952b7-b967-4bf2-b778-14baec6e3382 ANUMBER            4   
               ae12f15f-b1fd-4eca-9f20-b938e3509745 ANUMBER            4   
               bb8fdcc8-5735-42e8-a136-762f4407a6a5 ANUMBER            2   
               f60bb472-4afc-4903-836c-f8eacc686631 ANUMBER            4   

                                                                n_perfect_trials  \
participant_id session_uuid                         session_id                     
1025           89d0639e-75de-4b23-8a73-105e47608e52 2                          0   
               b214346e-66ce-4474-a413-9bd5ec992ec3 1                          1   
1053           87aafffd-97d7-4510-99b5-a3ae0de45e69 1                          0   
               d9b9baaf-3416-4677-a3e1-b421c8220c53 2                          2   
1059           3d621016-fe1c-4b3c-8858-7e2a7cf550a1 2                          0   
...                                                                          ...   
QUALTRICSID    69611119-0775-4984-be4c-0f82a2984495 ANUMBER                    3   
               796952b7-b967-4bf2-b778-14baec6e3382 ANUMBER                    3   
               ae12f15f-b1fd-4eca-9f20-b938e3509745 ANUMBER                    1   
               bb8fdcc8-5735-42e8-a136-762f4407a6a5 ANUMBER                    0   
               f60bb472-4afc-4903-836c-f8eacc686631 ANUMBER                    1   

                                                                mean_correct_dots  \
participant_id session_uuid                         session_id                      
1025           89d0639e-75de-4b23-8a73-105e47608e52 2                        0.75   
               b214346e-66ce-4474-a413-9bd5ec992ec3 1                        1.00   
1053           87aafffd-97d7-4510-99b5-a3ae0de45e69 1                        0.75   
               d9b9baaf-3416-4677-a3e1-b421c8220c53 2                        1.50   
1059           3d621016-fe1c-4b3c-8858-7e2a7cf550a1 2                        1.50   
...                                                                           ...   
QUALTRICSID    69611119-0775-4984-be4c-0f82a2984495 ANUMBER                  2.75   
               79695

Processing activity: symbol-search
-------------------------


flag_is_invalid_n_trials  \
participant_id session_uuid                         session_id                             
1025           2592d493-61b2-4129-97d7-90b373fbb9f2 1                              False   
               39fceb35-b433-49d0-acb2-b0a4801be038 2                              False   
1053           1ff76f21-9cf5-4d20-aaf5-01a7b6acb2c4 2                              False   
               5d2029a7-4a30-44ac-9e3f-54d0deb3ebe8 1                               True   
1059           3c80fb20-b91d-492b-9549-7a2de3718bcf 1                              False   
...                                                                                  ...   
QUALTRICSID    a26a9de9-b8ae-4326-8442-6de0272f6392 ANUMBER                        False   
               c4362e29-5107-42c9-9eee-a1986286b85f ANUMBER                        False   
               d38fd29f-055b-4ffb-8937-65a95e3e864f ANUMBER                        False   
               da915955-a948-4f6f-80ea-ffd8dc3c7a41 ANUMBER                        False   
               fbbd78fa-878f-4cf4-8fdd-51425c8055f0 ANUMBER                         True   

                                                                n_trials  \
participant_id session_uuid                         session_id             
1025           2592d493-61b2-4129-97d7-90b373fbb9f2 1                 20   
               39fceb35-b433-49d0-acb2-b0a4801be038 2                 20   
1053           1ff76f21-9cf5-4d20-aaf5-01a7b6acb2c4 2                 20   
               5d2029a7-4a30-44ac-9e3f-54d0deb3ebe8 1                 19   
1059           3c80fb20-b91d-492b-9549-7a2de3718bcf 1                 20   
...                                                                  ...   
QUALTRICSID    a26a9de9-b8ae-4326-8442-6de0272f6392 ANUMBER           20   
               c4362e29-5107-42c9-9eee-a1986286b85f ANUMBER           20   
               d38fd29f-055b-4ffb-8937-65a95e3e864f ANUMBER           20   
               da915955-a948-4f6f-80ea-ffd8dc3c7a41 ANUMBER           20   
               fbbd78fa-878f-4cf4-8fdd-51425c8055f0 ANUMBER            1   

                                                                n_trials_lure  \
participant_id session_uuid                         session_id                  
1025           2592d493-61b2-4129-97d7-90b373fbb9f2 1                      10   
               39fceb35-b433-49d0-acb2-b0a4801be038 2                      10   
1053           1ff76f21-9cf5-4d20-aaf5-01a7b6acb2c4 2                      10   
               5d2029a7-4a30-44ac-9e3f-54d0deb3ebe8 1                       9   
1059           3c80fb20-b91d-492b-9549-7a2de3718bcf 1                      10   
...                                                                       ...   
QUALTRICSID    a26a9de9-b8ae-4326-8442-6de0272f6392 ANUMBER                10   
               c4362e29-5107-42c9-9eee-a1986286b85f ANUMBER                10   
               d38fd29f-055b-4ffb-8937-65a95e3e864f ANUMBER                10   
               da915955-a948-4f6f-80ea-ffd8dc3c7a41 ANUMBER                10   
               fbbd78fa-878f-4cf4-8fdd-51425c8055f0 ANUMBER                 1   

                                                                n_correct_trials  \
participant_id session_uuid                         session_id                     
1025           2592d493-61b2-4129-97d7-90b373fbb9f2 1                         20   
               39fceb35-b433-49d0-acb2-b0a4801be038 2                         20   
1053           1ff76f21-9cf5-4d20-aaf5-01a7b6acb2c4 2                         18   
               5d2029a7-4a30-44ac-9e3f-54d0deb3ebe8 1                         17   
1059           3c80fb20-b91d-492b-9549-7a2de3718bcf 1                         18   
...                                                                          ...   
QUALTRICSID    a26a9de9-b8ae-4326-8442-6de0272f6392 ANUMBER                   19   
               c4362e29-5107-42c9-9eee-a1986286b85f ANUMBER         

flag_is_invalid_n_trials  \
participant_id session_uuid                         session_id                             
1025           2592d493-61b2-4129-97d7-90b373fbb9f2 1                              False   
               39fceb35-b433-49d0-acb2-b0a4801be038 2                              False   
1053           1ff76f21-9cf5-4d20-aaf5-01a7b6acb2c4 2                              False   
               5d2029a7-4a30-44ac-9e3f-54d0deb3ebe8 1                               True   
1059           3c80fb20-b91d-492b-9549-7a2de3718bcf 1                              False   
...                                                                                  ...   
QUALTRICSID    a26a9de9-b8ae-4326-8442-6de0272f6392 ANUMBER                        False   
               c4362e29-5107-42c9-9eee-a1986286b85f ANUMBER                        False   
               d38fd29f-055b-4ffb-8937-65a95e3e864f ANUMBER                        False   
               da915955-a948-4f6f-80ea-ffd8dc3c7a41 ANUMBER                        False   
               fbbd78fa-878f-4cf4-8fdd-51425c8055f0 ANUMBER                         True   

                                                                n_trials  \
participant_id session_uuid                         session_id             
1025           2592d493-61b2-4129-97d7-90b373fbb9f2 1                 20   
               39fceb35-b433-49d0-acb2-b0a4801be038 2                 20   
1053           1ff76f21-9cf5-4d20-aaf5-01a7b6acb2c4 2                 20   
               5d2029a7-4a30-44ac-9e3f-54d0deb3ebe8 1                 19   
1059           3c80fb20-b91d-492b-9549-7a2de3718bcf 1                 20   
...                                                                  ...   
QUALTRICSID    a26a9de9-b8ae-4326-8442-6de0272f6392 ANUMBER           20   
               c4362e29-5107-42c9-9eee-a1986286b85f ANUMBER           20   
               d38fd29f-055b-4ffb-8937-65a95e3e864f ANUMBER           20   
               da915955-a948-4f6f-80ea-ffd8dc3c7a41 ANUMBER           20   
               fbbd78fa-878f-4cf4-8fdd-51425c8055f0 ANUMBER            1   

                                                                n_trials_lure  \
participant_id session_uuid                         session_id                  
1025           2592d493-61b2-4129-97d7-90b373fbb9f2 1                      10   
               39fceb35-b433-49d0-acb2-b0a4801be038 2                      10   
1053           1ff76f21-9cf5-4d20-aaf5-01a7b6acb2c4 2                      10   
               5d2029a7-4a30-44ac-9e3f-54d0deb3ebe8 1                       9   
1059           3c80fb20-b91d-492b-9549-7a2de3718bcf 1                      10   
...                                                                       ...   
QUALTRICSID    a26a9de9-b8ae-4326-8442-6de0272f6392 ANUMBER                10   
               c4362e29-5107-42c9-9eee-a1986286b85f ANUMBER                10   
               d38fd29f-055b-4ffb-8937-65a95e3e864f ANUMBER                10   
               da915955-a948-4f6f-80ea-ffd8dc3c7a41 ANUMBER                10   
               fbbd78fa-878f-4cf4-8fdd-51425c8055f0 ANUMBER                 1   

                                                                n_correct_trials  \
participant_id session_uuid                         session_id                     
1025           2592d493-61b2-4129-97d7-90b373fbb9f2 1                         20   
               39fceb35-b433-49d0-acb2-b0a4801be038 2                         20   
1053           1ff76f21-9cf5-4d20-aaf5-01a7b6acb2c4 2                         18   
               5d2029a7-4a30-44ac-9e3f-54d0deb3ebe8 1                         17   
1059           3c80fb20-b91d-492b-9549-7a2de3718bcf 1                         18   
...                                                                          ...   
QUALTRICSID    a26a9de9-b8ae-4326-8442-6de0272f6392 ANUMBER                   19   
               c4362e29-5107-42c9-9eee-a1986286b85f ANUMBER         